In [111]:
import pandas as pd
import os

We use only the stocks in S&P 500 Index here for demonstration.

In [112]:
ticks_snp = pd.read_csv('../data/raw/S&P500_constituents.csv')['Symbol'].values

The raw data consists of .csv files, one for each stock, which contains market data such as $open, $close, $high, $low and $volume. We convert them into data frames containing all stocks of interest, one for each data type. Also, we take a limited time frame, Year 2018 to 2019 for the sake of training time in this demo.

In [181]:
in_path = '../data/raw/stocks'
out_path = '../data/processed'

for col in ['Open', 'Close', 'High', 'Low', 'Volume']:
    dfs = []
    for tick in ticks_snp: # Subset to stocks in S&P500
        file_name = tick + '.csv'
        file_path = os.path.join(in_path, file_name)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, index_col=0)
            # Subset to Year 2018-2019
            df_sub = df[[col]][(df.index > '2018') & (df.index < '2020')].round(2)
            df_sub.columns = [tick]
            dfs.append(df_sub)
    df_comb = pd.concat(dfs, axis=1).sort_index()
    df_comb = df_comb[sorted(df_comb.columns)].dropna(how='all')
    df_comb.to_csv(os.path.join(out_path, col + '.csv'))

We use the $open data to compute the daily forward return, which will be used a the response that we want to predict.

In [182]:
df_open = pd.read_csv(os.path.join(out_path, 'Open.csv'), index_col=0)
df_open_forward1 = df_open.shift(periods=-1)

In [183]:
forward_return = (df_open_forward1 / df_open - 1).dropna(how='all').round(4)
forward_return.to_csv(os.path.join(out_path, 'ForwardReturn.csv'))

In [187]:
df_1 = pd.read_csv('../data/processed/Open.csv', index_col=0)
df_2 = pd.read_csv('../data/processed/Close.csv', index_col=0)

In [188]:
df_2

,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WST,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,67.60,52.99,172.26,98.41,58.79,29.43,153.84,177.70,90.28,40.09,...,99.50,35.30,164.30,47.81,85.03,68.07,81.60,124.06,103.71,71.77
2018-01-03,69.32,52.34,172.23,99.95,58.92,29.46,154.55,181.04,91.40,39.78,...,99.62,35.61,162.52,47.49,86.70,68.90,81.53,124.92,105.77,72.10
2018-01-04,68.80,52.67,173.03,99.38,58.82,29.57,156.38,183.22,91.30,40.45,...,99.59,35.32,163.40,47.12,86.82,69.36,82.36,124.74,107.86,72.53
2018-01-05,69.90,52.65,175.00,101.11,58.99,29.45,157.67,185.34,91.67,40.18,...,100.84,35.27,164.49,46.79,86.75,69.23,82.84,125.98,109.54,73.36
2018-01-08,70.05,52.13,174.35,99.49,58.82,29.46,158.93,185.04,91.83,40.09,...,101.41,35.50,162.30,47.14,87.14,69.48,82.98,126.22,110.63,74.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,85.30,29.18,284.27,89.85,87.28,42.01,211.61,329.64,119.72,46.08,...,150.73,29.85,139.44,62.86,70.02,78.64,100.33,150.87,254.33,132.92
2019-12-26,85.45,29.67,289.91,89.83,87.28,42.18,212.05,331.20,119.73,46.20,...,150.77,29.99,140.94,62.79,70.13,78.60,101.79,150.30,254.42,133.03
2019-12-27,85.42,28.44,289.80,89.20,87.40,42.56,212.22,330.79,119.45,46.30,...,150.81,29.98,139.81,63.12,69.89,78.84,101.90,150.01,256.00,133.25


In [206]:
@jit(nopython=True)
def _ops_roll_std_arr(ar, window_size):
    '''
    Compute the rolling standard deviation per column of a NumPy array.
    
    Parameters:
        ar (np.ndarray): 2D Input array.
        window_size (int): Size of the rolling window.
        
    Returns:
        np.ndarray: 2D array of rolling standard deviations.
    '''
    n, d = ar.shape
    std_devs = []
    for i in range(n):
        win = ar[max(i+1-window_size, 0):(i+1)]
        win_std_devs = []
        for j in range(d):
            win_std_dev = np.std(win[:, j])
            win_std_devs.append(win_std_dev)
        std_devs.append(win_std_devs)
    return std_devs

def ops_roll_std(df: pd.DataFrame, window_size=5) -> pd.DataFrame:
    '''
    Compute the rolling standard deviation per column of a pandas DataFrame.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame.
        window_size (int): Size of the rolling window.
        
    Returns:
        pd.DataFrame: DataFrame of rolling standard deviations per column.
    '''
    ar = df.values
    ar_std_devs = _ops_roll_std_arr(ar, window_size)
    df_std_devs = pd.DataFrame(ar_std_devs, index=df.index, columns=df.columns)

    return df_std_devs

In [207]:
import time
# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
ops_roll_std(df_1)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
ops_roll_std(df_1)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

Elapsed (with compilation) = 0.6544778347015381
Elapsed (after compilation) = 0.03318500518798828


In [205]:
spearman_correlation(np.array([1,2,3]), np.array([1,2,3]))

1.0